# Lab Report "Demodulation" of Group 1n (Wörner, Velez, Northe)

The source code and the binaries from this report can be found in our Github [repository](https://github.com/NortheLo/Microwave_Eng_Lab/tree/main/Lab6_Analog_Digital_Mod).

## 7.1 Analog Modulation and Demodulation

Insert Report here

### 7.1.1 Generation of an Amplitude-Modulated Signal

Insert Report here

### 7.1.2 Demodulation of an Amplitude-Modulated Signal

The signals have the following properties:
- $f_c = 1 \text{MHz}$ carrier frequency (Frequency)
- $f_{FM} = 200 \text{kHz}$ frequency deviation (FM deviation)
- $f_s = 10 \text{kHz}$ signal frequency (low-frequency, modulating signal, FM rate)

**What frequency do you expect (frequency of the LF signal) and what frequency do you read off? Can you explain this effect?**

Insert Report here

**How and why can the amplifier help you with this?**

Insert Report here

Now create a frequency-modulated signal with the signal generator.
**Look at it with the spectrum analyzer and identify the components of the received signal. <br>
Record this with pictures. Compare the characteristics of the measured spectrum to them of the theoretical one.**

$\red{Insert \space screenshot}$

## 7.2 Digital Demodulation

The following settings were used:
- $ f_c = 800 \text{MHz} $
- $ P_{LO} = 10 \text{dBm} $ for the first signal generator (R%S in the script)
- $ P_{RF} = 0 \text{dBm}$ for the second signal generator (Agilent in the script)

**Which form of the IQ signals would you expect?**

Expectation: <br>
The expectation would be to see on each oscilloscope channel complementary binary values.

Think about how the modulated transmit
signal looks like. In addition, think about what an IQ demodulator does or how it is constructed.<br>
**Why could it be that the displayed IQ signals deviate from theory? What could help?**

IQ demodulator: <br>
- Splits signal in two
- mixes one branch (I) with the LO signal
- mixes the other (Q) with a 90° shifted signal
- Both branches get filtered to reduce alias effects

The deviation from the expectation is due to the fact that the RF signal (simulates signal from an antenna/TX) is not syncronized with the LO signal, which simulates the LO of a receiver. So the fix would be to syncronize the PLLs of our lab equipment, which can be done by hooking up the trigger of one signal generator to the other.

$\red{Insert \space screenshot}$

**Reconstruct the information signal according to (5.5) from the script (Calculate the
amplitude of the received signal. Take a photo or save a picture of the displayed signals.).**

$\red{Insert \space screenshot}$

**Explain why the I and Q signals changed after the LO frequency had been changed and
restored.**

Because the position of I and Q is only relative to each other, they can also be represented by a mirrowed position in the constellation diagram.

$\red{Pls \space correct \space if \space wrong}$